<a href="https://colab.research.google.com/github/Avinash4006/Sentiment-analysis-using-Machine-Learning-/blob/main/Sentiment_Analysis_of_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [60]:
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [61]:
from zipfile import ZipFile
data='/content/sentiment140.zip'
with ZipFile(data,'r') as zip:
  zip.extractall()
  print("Extracted dataset")

Extracted dataset


In [62]:
import numpy as np

In [63]:
from nltk.corpus import stopwords # NLP Toolkit library

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
!pip install scikit-learn


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [69]:
import nltk

In [70]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [72]:
#Data Preprocessing

In [73]:
!file -bi /content/training.1600000.processed.noemoticon.csv

text/csv; charset=utf-8


In [74]:
!pip install chardet

In [75]:
import pandas as pd

X_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [76]:
X_data.shape

(1599999, 6)

In [77]:
X_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [78]:
column_names=['Target','id','date','flag','user','text']
X_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [79]:
X_data.shape

(1600000, 6)

In [80]:
X_data.head()

,Target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [81]:
X_data.isnull()

,Target,id,date,flag,user,text
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
1599995,False,False,False,False,False,False
1599996,False,False,False,False,False,False
1599997,False,False,False,False,False,False
1599998,False,False,False,False,False,False


In [82]:
X_data.isnull().sum()

Target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [83]:
X_data['Target'].value_counts()

0    800000
4    800000
Name: Target, dtype: int64

In [84]:
X_data.replace({'Target':{4:1}},inplace=True)

In [85]:
X_data['Target'].value_counts() # 0 means Negative tweetsand 1 means positive tweets

0    800000
1    800000
Name: Target, dtype: int64

In [86]:
from nltk.stem.porter import PorterStemmer

In [87]:
#reducing the words into root word or keywords like actor,actress,acting=act

In [88]:
port_s=PorterStemmer()
import re #regular expression

In [89]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split() # split all the words into list
  stemmed_content=[port_s.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
X_data['Stemmed_content']=X_data['text'].apply(stemming)  # 1 hours to complete this execution

In [ ]:
X_data.head()

In [ ]:
print(X_data['Stemmed_content'])

In [ ]:
print(X_data['Target'])

In [ ]:
X=X_data['Stemmed_content'].values
Y=X_data['Target'].values

In [ ]:
print(X)

In [ ]:
print(Y) #separting data and label

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
vectorizer=TfidfVectorizer()
#convert the text into numerical data to train the ML model
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
#training the ML model
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy score of trainings data: ',training_data_accuracy)

In [ ]:
X_test_prediction=model.predict(X_test)
training_data_accuracy=accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('Accuracy score of test data: ',training_data_accuracy) #77.8%

In [ ]:
import pickle

In [ ]:
filename='Trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open('/content/Trained_model.sav','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('Negative tweet')
else:
  print('Positive tweet')

In [ ]:
X_new=X_test[3]
print(Y_test[3])
prediction=model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('Negative tweet')
else:
  print('Positive tweet')

In [ ]:
# Get user input
user_input = input("Enter a text: ")

# Test the model
model_prediction = model.predict(user_input)

# Display the results
print("Model prediction:", model_prediction)